In [ ]:
import os
import re
import zipfile
import json
import pandas as pd

In [4]:
flood_db_path = 'flood_db\\gfd_v1_4'

# Loads properties for each flood event
def load_flood_properties(file):
    with zipfile.ZipFile(file) as z:
        json_file = [file_name for file_name in z.namelist() if 'json' in file_name]
        with z.open(json_file[0]) as f:
            data = f.read()
            d = json.loads(data)
            d['fname'] = file
    return(d)

properties_list = [load_flood_properties(flood_db_path + '\\' + fname) for fname in os.listdir(flood_db_path)]
properties = pd.DataFrame(properties_list)
properties.to_csv('properties.tsv', sep = '\t')